In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
DATA_FILE = 'COVID-19.csv'

In [ ]:
df = pd.read_csv(DATA_FILE)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['確定日date'] = pd.to_datetime(df['確定日'])

In [ ]:
df.head()

In [ ]:
df.groupby('居住都道府県').count()['通し']

In [ ]:
pd.DataFrame(df.groupby(['確定日date']).count()['通し']).sort_values('確定日date')

In [ ]:
g_df = pd.DataFrame(df.groupby(['確定日date', '居住都道府県']).count()['通し']).sort_values(['居住都道府県', '確定日date'])
# g_df = pd.DataFrame(df.groupby(['確定日date', '居住都道府県'], as_index=False).count()['通し'])

In [ ]:
type(g_df)

In [ ]:
g_df.head()

In [ ]:
g_df = g_df.reset_index()

In [ ]:
ax = sns.lineplot(data=g_df, x='確定日date', y='通し', hue='居住都道府県')
ax.show()